In [1]:
import pandas as pd
#from pandas.io.parsers import ParserError
import numpy as np
from helper import get_mapper
import json
import os
import re
from sqlalchemy import create_engine

In [2]:
from os import listdir, stat
from os.path import isfile, join
BASE_DIR = "."
MIN_SIZE = 512

In [3]:
#bpm = pd.read_csv("../basic/block_plant_mapper.csv")
#bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [4]:
FOLDERS = [os.path.join(BASE_DIR, o) for o in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR,o))]
FOLDERS.sort()
FOLDERS = FOLDERS[1:-3]

In [5]:
bpm = pd.read_csv("../basic/plant_sse_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))
plantslist = list(set(bpm['plantid'].to_list()))

In [6]:
plantslist.remove('661-94')
plantslist.remove('661-10')

plantslist.remove('07-05-8290552')
plantslist.remove('06-05-100-0081105')
#plantslist.remove(

In [7]:
#refpl = "03-01-01241117210.csv"

In [8]:
#df = pd.read_csv("./combined/" + refpl)

In [9]:
#df

In [10]:
engine = create_engine('sqlite://', echo=False)

In [11]:
counter = 0
for plant in plantslist:
    #print(plant)
    try:
        print(plant)
        df = pd.read_csv("./combined/" + plant + ".csv")
        df['produced_at'] =  pd.to_datetime(df['produced_at'])
        melt = df.melt(id_vars='produced_at')
        melt.to_sql('power', con=engine, if_exists='append')
        counter += 1
        #print(df.shape)
    except FileNotFoundError:
        pass
print(counter)

666-12
06-09-661-0020-0001
661-88
06-05-112-0458158
661-38
14-20-60319980000
06-08-9840001
06-09-600-0025-0001
06-05-100-0989137
661-67
06-26200010633
661-19
13-50-138
661-54
661-91
661-112
661-43
661-12
661-111
666-13
06-09-500-0112-0001
06-09-674-2071-0001
06-05-900-0327252
06-05-300-9002708
03-07-07244141350
666-16
06-09-761-0026-0001
06-09-100-0003-0003
06-09-500-0354-0002
664-07
06-08-1479296
06-08-1574906
661-83
06-09-671-0140-0001
661-42
661-53
661-98
06-05-900-0021170
06-05-900-0888950
06-05-900-0205103
662-14
06-08-1383701
07-04-6048443
661-109
661-85
06-09-171-0002--C01
03-01-01010356620
06-05-700-9073419
12-50724070000
06-05-100-9021016
06-09-178-0722-0001
06-05-900-0001413
06-11-01-1105602
06-05-500-0342658
06-02-B10117A007
06-10-0075248
662-11
661-87
06-14600010611
06-00176010435
12-40716440000
06-05-900-9141660
661-64
661-113
06-04-11_2001178_8_0
06-11-01-1105609
03-01-01010959490
661-65
13-60-80407
662-06
12-40520430000
06-08-2948214
03-02-02221253020
06-09-261-0007-0041

In [12]:
CDF = pd.read_sql_table('power', engine)

In [13]:
#CDF

In [14]:
#CDF.drop(columns=['index'], inplace=True)
#CDF['produced_at'] =  pd.to_datetime(CDF['produced_at'])
CDF = CDF.set_index('produced_at') 

In [15]:
CDF["value"] = CDF["value"].astype(int)

In [16]:
#CDF

In [17]:
CDF2 = CDF.groupby('variable').resample('1M').sum()

# the inverse of groupby, reset_index
CDF3 = CDF2.reset_index()


/tmp/ipykernel_48029/2591619358.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CDF2 = CDF.groupby('variable').resample('1M').sum()
/tmp/ipykernel_48029/2591619358.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CDF2 = CDF.groupby('variable').resample('1M').sum()
/tmp/ipykernel_48029/2591619358.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CDF2 = CDF.groupby('variable').resample('1M').sum()
/tmp/ipykernel_4

In [18]:
CDF3

,variable,produced_at,index,value
0,BNA0140,2015-01-31,1319664,0
1,BNA0140,2015-02-28,1567668,0
2,BNA0140,2015-03-31,1975969,0
3,BNA0140,2015-04-30,2256108,0
4,BNA0140,2015-05-31,2658108,0
...,...,...,...,...
17954,SEE999585452499,2016-08-31,10219836,0
17955,SEE999585452499,2016-09-30,10200000,0
17956,SEE999585452499,2016-10-31,10917227,0
17957,SEE999585452499,2016-11-30,10846704,0


In [19]:
#gy = CDF.resample('1Y', on='produced_at').sum()
#gm = CDF.resample('1M', on='produced_at').sum()

In [20]:
gm2 = CDF3.reset_index(drop=True)
gm2['year'] = gm2['produced_at']
gm2['month'] = gm2['produced_at']

In [21]:
gm2['year'] = gm2['year'].apply(lambda x: str(x).split("-")[0])
gm2['month'] = gm2['month'].apply(lambda x: int(str(x).split("-")[1]))
gm2['month'] = gm2['month'].astype(int)

In [22]:
gm3 = gm2.sort_values(by=["year", 'month'])
gm4 = gm3.drop(columns='produced_at')

In [23]:
gm5 = gm4.reindex(columns=['year', "month", "variable", "value"])
gm6 = gm5.sort_values(by=["year", 'month'])

In [24]:
gm7 = gm6.rename(columns={"variable":"blockid", "value": "power"})
gm8 = gm7.reset_index(drop=True)

In [25]:
#gm5 = gm4.melt(id_vars=["year", "month"], var_name='blockid', value_name='power')
#gm5['power'] = gm5['power'].astype(int)

In [26]:
gm8

,year,month,blockid,power
0,2015,1,BNA0140,0
1,2015,1,BNA0143,0
2,2015,1,BNA0145,0
3,2015,1,BNA0146,136961
4,2015,1,BNA0215,0
...,...,...,...,...
17954,2022,12,SEE996720450723,28162
17955,2022,12,SEE997719859992,54197
17956,2022,12,SEE998199911088,465202
17957,2022,12,SEE998383491525,0


In [27]:
gm7

,year,month,blockid,power
0,2015,1,BNA0140,0
96,2015,1,BNA0143,0
192,2015,1,BNA0145,0
276,2015,1,BNA0146,136961
576,2015,1,BNA0215,0
...,...,...,...,...
17550,2022,12,SEE996720450723,28162
17646,2022,12,SEE997719859992,54197
17742,2022,12,SEE998199911088,465202
17838,2022,12,SEE998383491525,0


In [28]:
bpm

,plantid,sseid
0,01-50-01000004838,SEE957285549260
1,01-50-01000004838,SEE990438768378
2,01-50-01000004838,SEE966920434281
3,01-50-01000004838,SEE999539592561
4,01-50-01000004838,BNA0273
...,...,...
940,06-10-00101000,SEE943764365964
941,01-50-04000082392,SEE953266686481
942,01-50-04000082392,SEE934591835126
943,01-50-04000082392,SEE961021885321


In [29]:
pm1 = gm7.merge(bpm, left_on="blockid", right_on="sseid")
pm2 = pm1.drop('sseid', axis=1)

In [30]:
pm2

,year,month,blockid,power,plantid
0,2015,1,BNA0140,0,06-04-11_2001178_8_0
1,2015,2,BNA0140,0,06-04-11_2001178_8_0
2,2015,3,BNA0140,0,06-04-11_2001178_8_0
3,2015,4,BNA0140,0,06-04-11_2001178_8_0
4,2015,5,BNA0140,0,06-04-11_2001178_8_0
...,...,...,...,...,...
16296,2022,8,SEE913516809497,463120,06-05-500-0915123
16297,2022,9,SEE913516809497,334082,06-05-500-0915123
16298,2022,10,SEE913516809497,259657,06-05-500-0915123
16299,2022,11,SEE913516809497,536936,06-05-500-0915123


In [31]:
pm2['plantpower'] = pm2.groupby(['plantid', 'month', 'year'])['power'].transform('sum')

In [32]:
pm3 = pm2.drop_duplicates(['month', 'year', 'plantid'])
pm4 = pm3.drop(['blockid', 'power'], axis=1)

In [33]:
ym1 = pm4.copy()
ym1['yearpower'] = ym1.groupby(['plantid', 'year'])['plantpower'].transform('sum')
ym2 = ym1.drop_duplicates(['year', 'plantid'])
ym3 = ym2.drop(['month', 'plantpower'], axis=1)
ym4 = ym3.reset_index(drop=True)

In [34]:
ym4.loc[ym4.plantid == "06-05-100-0248923"]

,year,plantid,yearpower
277,2015,06-05-100-0248923,15389371
278,2016,06-05-100-0248923,21816032
279,2017,06-05-100-0248923,22612318
280,2018,06-05-100-0248923,25510420
281,2019,06-05-100-0248923,17882517
282,2020,06-05-100-0248923,14625943
283,2021,06-05-100-0248923,16556991
284,2022,06-05-100-0248923,21859813


In [40]:
ym4.loc[ym4.plantid == "06-05-500-0915123"]

,year,plantid,yearpower
525,2017,06-05-500-0915123,1
526,2018,06-05-500-0915123,0
527,2019,06-05-500-0915123,1495
528,2020,06-05-500-0915123,2275756
529,2021,06-05-500-0915123,4696684
530,2022,06-05-500-0915123,4558556


In [35]:
ym4.to_csv("yearly.csv", index=False)
ym4.to_csv("yearly_pg.csv", header=False)

In [36]:
gm8.to_csv("monthly.csv", header=False)
#gm7.to_csv("yearly_pg.csv")

In [37]:
#invCDF = CDF.pivot(columns='variable')

In [38]:
gm8

,year,month,blockid,power
0,2015,1,BNA0140,0
1,2015,1,BNA0143,0
2,2015,1,BNA0145,0
3,2015,1,BNA0146,136961
4,2015,1,BNA0215,0
...,...,...,...,...
17954,2022,12,SEE996720450723,28162
17955,2022,12,SEE997719859992,54197
17956,2022,12,SEE998199911088,465202
17957,2022,12,SEE998383491525,0


In [39]:
bpm

,plantid,sseid
0,01-50-01000004838,SEE957285549260
1,01-50-01000004838,SEE990438768378
2,01-50-01000004838,SEE966920434281
3,01-50-01000004838,SEE999539592561
4,01-50-01000004838,BNA0273
...,...,...
940,06-10-00101000,SEE943764365964
941,01-50-04000082392,SEE953266686481
942,01-50-04000082392,SEE934591835126
943,01-50-04000082392,SEE961021885321
